[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M11_Intro_To_Unsuerpvised_Learning/blob/master/11_1_Intro_to_Unsupervised_Learning_I.ipynb)

# Intro to Unsupervised Learning

## Text Retrieval Problem

### Your name :     

### Data Link :
* [DBpedia](https://wiki.dbpedia.org/)
* [Dataset](https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv)

#### 실습목표 I<br>
1. Bag of Words, TF-IDF를 이해한다.
2. Scikit-Learn을 이용하여 Retrieval을 할 줄 안다.

#### 실습목표 II<br>
1. 문제의 구조를 정교화 하기 위해 발전한 과정의 흐름을 안다.
---------------
Rayleigh Kim @ D:plus


In [139]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
'''
사이킷런의 라이브러리들을 미리 불러두는 것도 좋지만,
세부적인 메쏘드가 너무 많아서 필요한것만 불러오도록 할 것이다.
'''


'\n사이킷런의 라이브러리들을 미리 불러두는 것도 좋지만,\n세부적인 메쏘드가 너무 많아서 필요한것만 불러오도록 할 것이다.\n'

## Load and Prepare the data

In [0]:
data_path = 'https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv'
people = pd.read_csv(data_path)

In [141]:
len(people)

59071

In [142]:
people.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


우리는 URL은 안쓸꺼니까! 날려버리자.

왜인지 URI로 되어 있다.

In [143]:
if 'URI' in list(people):
    people.drop(labels=['URI'], axis=1, inplace=True)
people.head()

,name,text
0,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## Explore the dataset

엘튼 존! 조지 클루니!

In [144]:
elton = people.loc[people['name']=='Elton John']
elton

,name,text
19923,Elton John,sir elton hercules john cbe born reginald kenn...


In [145]:
list(elton['text'])

['sir elton hercules john cbe born reginald kenneth dwight 25 march 1947 is an english singer songwriter composer pianist record producer and occasional actor he has worked with lyricist bernie taupin as his songwriter partner since 1967 they have collaborated on more than 30 albums to datein his fivedecade career elton john has sold more than 300 million records making him one of the bestselling music artists in the world he has more than fifty top 40 hits including seven consecutive no 1 us albums 58 billboard top 40 singles 27 top 10 four no 2 and nine no 1 for 31 consecutive years 19702000 he had at least one song in the billboard hot 100 his single something about the way you look tonightcandle in the wind 1997 sold over 33 million copies worldwide and is the bestselling single of all time he has received six grammy awards five brit awards winning two awards for outstanding contribution to music and the first brits icon in 2013 for his lasting impact on british culture an academy 

In [146]:
george = people.loc[people['name'].str.contains('George')]
george['name']

213                        George Brent Mickum IV
586                          George Athan Billias
977                            George M. Galambos
1100                                  George Azar
1323                                George Quasha
1402                       George de la Pe%C3%B1a
1418                                George E. Fox
1524                                Willie George
1710                                George Athans
2486                             George Halvorson
2654                               George Gabriel
2657                              Billy Georgette
2770                  George McMahon (politician)
2969                              George Brancato
3883                               George Kurdahi
4904                      George Hart (physicist)
5180                           George P. Fletcher
5624                                  George Piro
5918                             George P. Taylor
5952                                  George Roll


In [147]:
clooney = george.loc[george['name'].str.contains('Clooney')]
clooney

,name,text
38514,George Clooney,george timothy clooney born may 6 1961 is an a...


In [148]:
list(clooney['text'])

['george timothy clooney born may 6 1961 is an american actor writer producer director and activist he has received three golden globe awards for his work as an actor and two academy awards one for acting and the other for producingclooney made his acting debut on television in 1978 and later gained wide recognition in his role as dr doug ross on the longrunning medical drama er from 1994 to 1999 for which he received two emmy award nominations while working on er he began attracting a variety of leading roles in films including the superhero film batman robin 1997 and the crime comedy out of sight 1998 in which he first worked with a director who would become a longtime collaborator steven soderbergh in 1999 clooney took the lead role in three kings a wellreceived war satire set during the gulf warin 2001 clooneys fame widened with the release of his biggest commercial success the heist comedy oceans eleven the first of the film trilogy a remake of the 1960 film with frank sinatra as 

## Bag of Words Simple ver

텍스트 문서들은 있다.<br>

텍스트들을 단어 등장여부를 이용한 binary vector로 만들자.

In [0]:
'''
Library Loading
'''
from collections import Counter, OrderedDict
import sklearn.feature_extraction.text as text

### 문장 하나를 사전으로 만들어 보자.

In [0]:
def Ordered_words(id = 0):
    sample_string = people['text'][id]
    string_splitted = sample_string.split()
    word_dict = dict(Counter(string_splitted))
    word_dict_list = [(k,v) for k, v in word_dict.items()]
    word_dict_list.sort(key = lambda x:x[1], reverse=True)
    print('name : ',people['name'][id])
    print(sample_string)
    print(word_dict_list)
    

In [151]:
'''
Ordered_words(n)
n 을 바꾸어 가며 확인해보자.
'''

Ordered_words(3)

name :  Franz Rottensteiner
franz rottensteiner born in waidmannsfeld lower austria austria on 18 january 1942 is an austrian publisher and critic in the fields of science fiction and the fantasticrottensteiner studied journalism english and history at the university of vienna receiving his doctorate in 1969 he served about fifteen years as librarian and editor at the sterreichisches institut fr bauforschung in vienna in addition he produced a number of translations into german of leading sf authors including herbert w franke stanislaw lem philip k dick kobo abe cordwainer smith brian w aldiss and the strugatski brothersin 1973 his new york anthology view from another shore of european science fiction introduced a number of continental authors to the englishreading public some of the authors in the work are stanislaw lem josef nesvadba gerard klein and jeanpierre andrevonthe year 1975 saw the start of his series die phantastischen romane for seven years it republished works of both les

### 모든 텍스트를 이용하여, 존재하는 단어들의 사전을 만들어 보자!

In [0]:
word_bin = text.CountVectorizer(binary = True)

In [0]:
X_simple = word_bin.fit_transform(people['text'])

In [154]:
type(X_simple)

scipy.sparse.csr.csr_matrix

In [155]:
print(len(word_bin.vocabulary_))
word_bin.vocabulary_

548429


{'digby': 160652,
 'morrell': 336162,
 'born': 96612,
 '10': 704,
 'october': 359577,
 '1979': 11143,
 'is': 259893,
 'former': 203171,
 'australian': 72887,
 'rules': 427313,
 'footballer': 202230,
 'who': 530685,
 'played': 386988,
 'with': 533439,
 'the': 488148,
 'kangaroos': 272528,
 'and': 58906,
 'carlton': 111950,
 'in': 251905,
 'football': 202189,
 'league': 293515,
 'aflfrom': 48415,
 'western': 529346,
 'australia': 72734,
 'his': 239271,
 'early': 172662,
 'senior': 442836,
 'for': 202462,
 'west': 529244,
 'perth': 380905,
 '44game': 34683,
 'career': 111212,
 'falcons': 191106,
 'spanned': 461891,
 '19982000': 16893,
 'he': 233774,
 'was': 525845,
 'clubs': 128392,
 'leading': 293448,
 'goalkicker': 218903,
 '2000': 18132,
 'at': 70622,
 'age': 49200,
 'of': 360068,
 '21': 28353,
 'recruited': 412470,
 'to': 494541,
 'by': 105803,
 'club': 128164,
 'its': 261721,
 'third': 490181,
 'round': 425753,
 'selection': 441507,
 '2001': 18661,
 'afl': 48375,
 'rookie': 424587,
 

### 문장 하나를 벡터로 만들어 보자!

 모든 문장을 이용하여 단어 사전을 만들어 두었다!
 
 이제, 문장 하나를 단어 존재 여부를 이용하여 벡터로 만들 수 있다.

In [0]:
def simple_BOW_vector(id=0):
    sample_array = word_bin.transform([people['text'][id]]).toarray()[0]
    print('array 사이즈 : ', len(sample_array))
    print('등장한 단어의 개수 : ', sum(sample_array==1))
    print(sample_array)

In [157]:
simple_BOW_vector(4)

array 사이즈 :  548429
등장한 단어의 개수 :  113
[0 0 0 ... 0 0 0]


### 문장들을 벡터로 만들어서 데이터 프레임에 붙일 수 있을까? -> 붙이지 말자!

정말 궁금한 사람들은 아래의 코드를 실행해보자.

메모리가 터지는 것을 관찰할 수 있다!

```
word_bin.transform(people['text'].toarray())
```

위에서 아래와 같은 코드를 사용했었다.

```
X_simple = word_bin.fit_transform(people['text'])
```

X_simple는 scipy.sparse.csr.csr_matrix 라고 위에서 확인했었는데,  사실 X가 우리가 원하는 그 메트릭스다!

다만, 사이즈가 54만이 넘어가는 벡터에서 고작 단어 몇백개의 위치만 1이 되는 array를 그대로 생성하는 것은 메모리가 괴로워하기 때문에, 우리는 잘 압축된 메트릭스를 대신하여 사용한다.

아래의 셀을 실행해보면 어떻게 된 것인지 감을 잡을 수 있다!

In [158]:
people.head(1)

,name,text
0,Digby Morrell,digby morrell born 10 october 1979 is a former...


In [159]:
simple_BOW_vector(0)

array 사이즈 :  548429
등장한 단어의 개수 :  138
[0 0 0 ... 0 0 0]


In [160]:
X_simple[0,:].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [161]:
X_simple[0,:].todense().sum()

138

In [162]:
print(X_simple[0,:])

  (0, 323069)	1
  (0, 130972)	1
  (0, 373306)	1
  (0, 174843)	1
  (0, 383407)	1
  (0, 483084)	1
  (0, 147491)	1
  (0, 174268)	1
  (0, 55769)	1
  (0, 129425)	1
  (0, 128951)	1
  (0, 232352)	1
  (0, 27213)	1
  (0, 451997)	1
  (0, 395043)	1
  (0, 164271)	1
  (0, 395008)	1
  (0, 25312)	1
  (0, 163854)	1
  (0, 184523)	1
  (0, 470394)	1
  (0, 26651)	1
  (0, 511222)	1
  (0, 23789)	1
  (0, 207224)	1
  :	:
  (0, 239271)	1
  (0, 72734)	1
  (0, 529346)	1
  (0, 48415)	1
  (0, 293515)	1
  (0, 202189)	1
  (0, 251905)	1
  (0, 111950)	1
  (0, 58906)	1
  (0, 272528)	1
  (0, 488148)	1
  (0, 533439)	1
  (0, 386988)	1
  (0, 530685)	1
  (0, 202230)	1
  (0, 427313)	1
  (0, 72887)	1
  (0, 203171)	1
  (0, 259893)	1
  (0, 11143)	1
  (0, 359577)	1
  (0, 704)	1
  (0, 96612)	1
  (0, 336162)	1
  (0, 160652)	1


### 우리는 sparse_matrix를 이용하여 그대로 nearest neighbors 도 해보고 클러스터링도 해볼 것이다!

NearestNeighbor search를 해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_simple = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_simple)

In [0]:
def nearest_search(name, nbrs = nbrs_simple, X = X_simple):
    queried_id = people.index[people['name']==name]
    queried_array = X[queried_id, :]
    distances, indices = nbrs.kneighbors(queried_array)
    temp_df = people.loc[indices.tolist()[0]]
    temp_df['rank'] = [0,1,2,3,4,5]
    distances = [ '{:.6f}'.format(elem) for elem in distances.tolist()[0]]
    temp_df['distances'] = distances
#     print(temp_df)
    return(temp_df)
    

In [166]:
nearest_search(name = 'Angelina Jolie', nbrs = nbrs_simple, X = X_simple).head(3)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
24426,Brad Pitt,william bradley brad pitt born december 18 196...,1,0.643529
44992,Julianne Moore,julianne moore born julie anne smith december ...,2,0.660615


In [167]:
nearest_search(name = 'Victoria Beckham', nbrs = nbrs_simple, X = X_simple).head(3)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
55821,Cheryl Cole,cheryl ann fernandezversini ne tweedy formerly...,1,0.664117
44581,Heidi Klum,heidi klum pronounced hadi klm born june 1 197...,2,0.682303


## Bag of Words Count ver

이제는 Bag of Words를 단어 수로 구현해보자!

In [0]:
'''
Library Loading
'''

import sklearn.feature_extraction.text as text

위에서는 아래와 같은 코드를 사용했다
```
word_bin = text.CountVectorizer(binary = True)
```

binary = True 가 되면, 단어의 등장 여부를 가지고 vector를 만들지만,

binary = False로 해두면, 단어의 등장 빈도를 이용하여 vector를 만든다.

In [0]:
word_count = text.CountVectorizer(binary = False)
X_count = word_count.fit_transform(people['text'])

In [170]:
people.head(1)

,name,text
0,Digby Morrell,digby morrell born 10 october 1979 is a former...


In [171]:
simple_BOW_vector(0)

array 사이즈 :  548429
등장한 단어의 개수 :  138
[0 0 0 ... 0 0 0]


In [172]:
X_count[0,:].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [173]:
print('총 단어 등장 수 :', X_count[0,:].todense().sum())
print('등장한 단어의 수 :', X_simple[0,:].todense().sum())

총 단어 등장 수 : 247
등장한 단어의 수 : 138


Bag of Words 가  등장여부에서 단어횟수로 더 정교화 되었다!

Nearest Neighbor search해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_count = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_count)

In [243]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_count, X = X_count)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
57434,Glenn Close,glenn close born march 19 1947 is an american ...,1,0.198190
57954,Cher,cher r born cherilyn sarkisian may 20 1946 is ...,2,0.206035
34756,Maggie Smith,dame margaret natalie maggie smith ch dbe born...,3,0.207389
55582,Rani Mukerji,rani mukerji rni mkhrdi born 21 march 1978 is ...,4,0.214426
44992,Julianne Moore,julianne moore born julie anne smith december ...,5,0.215354


In [242]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_simple, X = X_simple)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
24426,Brad Pitt,william bradley brad pitt born december 18 196...,1,0.643529
44992,Julianne Moore,julianne moore born julie anne smith december ...,2,0.660615
29562,Felicity Huffman,felicity kendall huffman born december 9 1962 ...,3,0.661333
21644,Jodie Foster,alicia christian foster born november 19 1962 ...,4,0.671561
38514,George Clooney,george timothy clooney born may 6 1961 is an a...,5,0.673001


In [244]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_count, X = X_count)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
669,Mary Fitzgerald (artist),mary fitzgerald born 1956 is an irish artist w...,1,0.211543
45129,Adrienne Corri,adrienne corri born 13 november 1931 glasgow s...,2,0.218543
39504,Beverly Jane Fry,beverly jane fry is an australian ballerina bo...,3,0.221893
13937,Raman Mundair,raman mundair is a british poet writer artist ...,4,0.222449
48867,Jacqueline de Jong,jacqueline de jong born 1939 is a dutch painte...,5,0.222511


In [245]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_simple, X = X_simple)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
55821,Cheryl Cole,cheryl ann fernandezversini ne tweedy formerly...,1,0.664117
44581,Heidi Klum,heidi klum pronounced hadi klm born june 1 197...,2,0.682303
5168,Simon Fuller,simon fuller born 17 may 1960 is an english en...,3,0.701995
27793,Adele,adele laurie blue adkins mbe born 5 may 1988 k...,4,0.704347
17264,Mel B,melanie janine brown born 29 may 1975 better k...,5,0.706237


## 이제는 TF-IDF 다!

다시 한 번 TF와 IDF를 복습하고 오자

In [0]:
'''
Library Loading
'''

import sklearn.feature_extraction.text as text

이 [문서](https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer) 를 참고하면 max_df와 min_df 에 대한 이해를 할 수 있다

In [0]:
word_tfidf = text.TfidfVectorizer(max_df = 0.5)
X_tfidf = word_tfidf.fit_transform(people['text'])

잠시 idf 를 감상하자!

In [0]:
idf = word_tfidf.idf_

In [254]:
'''
단어 인덱스를 보자
'''
print(len(word_tfidf.vocabulary_))
word_tfidf.vocabulary_


548408


{'digby': 160645,
 'morrell': 336148,
 '10': 704,
 'october': 359563,
 '1979': 11143,
 'former': 203163,
 'australian': 72882,
 'rules': 427297,
 'footballer': 202223,
 'who': 530665,
 'played': 386972,
 'kangaroos': 272514,
 'carlton': 111943,
 'football': 202182,
 'league': 293501,
 'aflfrom': 48415,
 'western': 529326,
 'australia': 72729,
 'early': 172655,
 'senior': 442820,
 'west': 529224,
 'perth': 380889,
 '44game': 34683,
 'career': 111205,
 'falcons': 191099,
 'spanned': 461875,
 '19982000': 16893,
 'clubs': 128385,
 'leading': 293434,
 'goalkicker': 218894,
 '2000': 18132,
 'age': 49200,
 '21': 28353,
 'recruited': 412454,
 'club': 128157,
 'its': 261707,
 'third': 490163,
 'round': 425737,
 'selection': 441491,
 '2001': 18661,
 'afl': 48375,
 'rookie': 424571,
 'draft': 168041,
 'forward': 203643,
 'twice': 504962,
 'kicked': 277949,
 'five': 199440,
 'goals': 218921,
 'during': 171490,
 'time': 492499,
 'first': 198980,
 'losing': 304310,
 'cause': 114347,
 'against': 4909

In [255]:
'''
scikit learn은 기본적으로 idf에 1을 더해두었다.
max_df = 0.5 를 둘 경우, 총 문서의 50% 이상의 문서들에 출현하는 단어는
다 제외해 버린다!

당연히 the는 에러가 남
'''
id = word_tfidf.vocabulary_['the']
idf[id]

KeyError: ignored

In [232]:
id = word_tfidf.vocabulary_['george']
idf[id]

3.795880636958985

In [233]:
id = word_tfidf.vocabulary_['metal']
idf[id]

5.584595121135338

0번 텍스트에 대해서 서로의 차이점을 다시 확인해보자.

* Bag of Words, Binary ver
* Bag of Words, Count ver
* TF-IDF

In [0]:
print(X_simple[0,:])


In [0]:
print(X_count[0, :])

In [0]:
print(X_tfidf[0, :])

이제, tf-idf를 이용하여 nearest neighbor search 해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_tfidf = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_tfidf)

안젤리나 졸리 비교 최종판!

In [261]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_tfidf, X = X_tfidf)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
29009,Barbara Hershey,barbara hershey born barbara lynn herzstein fe...,1,0.665955
57434,Glenn Close,glenn close born march 19 1947 is an american ...,2,0.691701
54362,Konkona Sen Sharma,konkona sen sharma bengali born 3 december 197...,3,0.697263
21644,Jodie Foster,alicia christian foster born november 19 1962 ...,4,0.699095
44992,Julianne Moore,julianne moore born julie anne smith december ...,5,0.702313


In [262]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_count, X = X_count)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
57434,Glenn Close,glenn close born march 19 1947 is an american ...,1,0.198190
57954,Cher,cher r born cherilyn sarkisian may 20 1946 is ...,2,0.206035
34756,Maggie Smith,dame margaret natalie maggie smith ch dbe born...,3,0.207389
55582,Rani Mukerji,rani mukerji rni mkhrdi born 21 march 1978 is ...,4,0.214426
44992,Julianne Moore,julianne moore born julie anne smith december ...,5,0.215354


In [263]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_simple, X = X_simple)

,name,text,rank,distances
39521,Angelina Jolie,angelina jolie doli johlee born angelina jolie...,0,0.000000
24426,Brad Pitt,william bradley brad pitt born december 18 196...,1,0.643529
44992,Julianne Moore,julianne moore born julie anne smith december ...,2,0.660615
29562,Felicity Huffman,felicity kendall huffman born december 9 1962 ...,3,0.661333
21644,Jodie Foster,alicia christian foster born november 19 1962 ...,4,0.671561
38514,George Clooney,george timothy clooney born may 6 1961 is an a...,5,0.673001


빅토리아 베컴 비교 최종판!

In [264]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_tfidf, X = X_tfidf)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
23386,David Beckham,david robert joseph beckham obe bkm born 2 may...,1,0.582914
17264,Mel B,melanie janine brown born 29 may 1975 better k...,2,0.791186
39144,Stephen Dow Beckham,stephen dow beckham is a noted american histor...,3,0.803588
58438,Mona al Mansouri,mona al mansouri arabic is an international de...,4,0.804467
57583,Caroline Rush,caroline rush cbe is a marketing professional ...,5,0.804482


In [265]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_count, X = X_count)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
669,Mary Fitzgerald (artist),mary fitzgerald born 1956 is an irish artist w...,1,0.211543
45129,Adrienne Corri,adrienne corri born 13 november 1931 glasgow s...,2,0.218543
39504,Beverly Jane Fry,beverly jane fry is an australian ballerina bo...,3,0.221893
13937,Raman Mundair,raman mundair is a british poet writer artist ...,4,0.222449
48867,Jacqueline de Jong,jacqueline de jong born 1939 is a dutch painte...,5,0.222511


In [266]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_simple, X = X_simple)

,name,text,rank,distances
50411,Victoria Beckham,victoria caroline beckham ne adams born 17 apr...,0,0.000000
55821,Cheryl Cole,cheryl ann fernandezversini ne tweedy formerly...,1,0.664117
44581,Heidi Klum,heidi klum pronounced hadi klm born june 1 197...,2,0.682303
5168,Simon Fuller,simon fuller born 17 may 1960 is an english en...,3,0.701995
27793,Adele,adele laurie blue adkins mbe born 5 may 1988 k...,4,0.704347
17264,Mel B,melanie janine brown born 29 may 1975 better k...,5,0.706237


## 클러스터링도 해봐야지!

DBSCAN을 해보자!

그리고, 감상하자.

메모리가 터지는 것을!

In [0]:
'''
라이브러리 로딩
'''

from sklearn.cluster import DBSCAN
from sklearn import metrics

In [0]:
dbscan_simple = DBSCAN(eps = 0.4, min_samples = 4, metric = 'cosine').fit(X_simple)

In [0]:
dbscan_simple.labels_